### Initial Set-Up

In [16]:
import os
from pathlib import Path
import alpaca_trade_api as tradeapi
import pandas as pd
import numpy as np
import datetime
import time
from dotenv import load_dotenv


In [17]:
# Load .env enviroment variables
load_dotenv()

True

In [18]:
# Set Alpaca API key and secret
alpaca_api_key = os.getenv("ALPACA_API_KEY")
alpaca_secret_key = os.getenv("ALPACA_SECRET_KEY")
print(alpaca_api_key)

PKXGKRUFDY45SB8IVZ0C


In [19]:
# Create the Alpaca API object, specifying use of the paper trading account:
ALPACA_API_BASE_URL = "https://paper-api.alpaca.markets"
api = tradeapi.REST(alpaca_api_key, alpaca_secret_key,ALPACA_API_BASE_URL, api_version='v2')

### Data Generation



#### 1. Create a ticker list, beginning and end dates, and timeframe interval.


In [20]:
# Define a list of tickers
ticker_list = ['FB','AMZN','AAPL','NFLX', 'GOOGL', 'MSFT', 'TSLA']
# declare begin and end date strings
beg_date = '2021-01-05'
end_date = '2021-01-05'
# we convert begin and end date to formats that the ALPACA API requires
start =  pd.Timestamp(f'{beg_date} 09:30:00-0400', tz='America/New_York').replace(hour=9, minute=30, second=0).astimezone('GMT').isoformat()[:-6]+'Z'
end   =  pd.Timestamp(f'{end_date} 16:00:00-0400', tz='America/New_York').replace(hour=16, minute=0, second=0).astimezone('GMT').isoformat()[:-6]+'Z'
# We set the time frequency at which we want to pull prices
timeframe='1Min'

#### 2. Ping the Alpaca API for the data and store it in a DataFrame called `prices` by using the `get_barset` function combined with the `df` method from the Alpaca Trade SDK.

In [21]:
# Pull prices from the ALPACA API
prices = api.get_barset(ticker_list, timeframe,limit=1000, start=start, end=end).df

#### 3. Store only the close prices from the `prices` DataFrame in a new DataFrame called `df_closing_prices`, then view the head and tail to confirm the following:
* First price for each stock on the open at 9:30 Eastern Time.
* Last price for the day on the close at 3:59 pm Eastern Time.

In [22]:
# Create an empty DataFrame for closing prices
df_closing_prices = pd.DataFrame()

# Fetch the closing prices for each one of the tickers and store in a column named after that ticker
df_closing_prices["FB"] = prices["FB"]["close"]
df_closing_prices["AMZN"] = prices["AMZN"]["close"]
df_closing_prices["AAPL"] = prices["AAPL"]["close"]
df_closing_prices["NFLX"] = prices["NFLX"]["close"]
df_closing_prices["GOOGL"] = prices["GOOGL"]["close"]
df_closing_prices['MSFT'] = prices['MSFT']["close"]
df_closing_prices['TSLA'] = prices['TSLA']["close"]

In [23]:
# Preview first five rows
df_closing_prices.head()

,FB,AMZN,AAPL,NFLX,GOOGL,MSFT,TSLA
time,,,,,,,
2021-01-05 09:30:00-05:00,269.00,3172.980,129.485,521.030,1724.17,217.65,725.23
2021-01-05 09:31:00-05:00,269.17,3177.810,130.060,521.365,1724.05,217.63,723.00
2021-01-05 09:32:00-05:00,269.72,3175.470,130.020,520.770,1721.61,217.77,720.57
2021-01-05 09:33:00-05:00,268.80,3179.360,130.120,520.000,NaN,217.72,719.71
2021-01-05 09:34:00-05:00,269.58,3184.015,130.510,522.240,1720.30,217.31,724.22


In [24]:
# Preview last five rows
df_closing_prices.tail()

,FB,AMZN,AAPL,NFLX,GOOGL,MSFT,TSLA
time,,,,,,,
2021-01-05 15:56:00-05:00,270.65,3219.84,130.850,519.57,1738.15,217.970,733.00
2021-01-05 15:57:00-05:00,270.91,3222.70,131.010,520.46,1738.99,218.175,734.49
2021-01-05 15:58:00-05:00,270.88,3221.18,130.990,520.30,1738.84,218.150,734.83
2021-01-05 15:59:00-05:00,270.86,3219.67,130.965,520.76,1740.57,218.000,735.33
2021-01-05 16:00:00-05:00,NaN,NaN,131.140,NaN,NaN,NaN,NaN


#### 4. When viewing the head and tail, you'll notice several `NaN` values.
* Alpaca reports `NaN` for minutes without any trades occuring as missing.
* These values must be removed, we use Panda's `ffill()` function to "forward fill", or replace, those prices with the previous values (since the price has not changed).


In [25]:
# Use Pandas' forward fill function to fill missing values (be sure to set inplace=True)
df_closing_prices.ffill(inplace=True)   
df_closing_prices.head()

,FB,AMZN,AAPL,NFLX,GOOGL,MSFT,TSLA
time,,,,,,,
2021-01-05 09:30:00-05:00,269.00,3172.980,129.485,521.030,1724.17,217.65,725.23
2021-01-05 09:31:00-05:00,269.17,3177.810,130.060,521.365,1724.05,217.63,723.00
2021-01-05 09:32:00-05:00,269.72,3175.470,130.020,520.770,1721.61,217.77,720.57
2021-01-05 09:33:00-05:00,268.80,3179.360,130.120,520.000,1721.61,217.72,719.71
2021-01-05 09:34:00-05:00,269.58,3184.015,130.510,522.240,1720.30,217.31,724.22
